In [1]:
import sys
sys.path.append( '../relative_fitness_mechanisms/' )
import plot_utils
from relative_fitness_mechanisms import RelativeFitnessGP, SquaredExpCov

ModuleNotFoundError: No module named 'relative_fitness_mechanisms'

In [ ]:
import jax
from jax import jit, lax
from functools import partial
import jax.numpy as jnp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import evofr as ef

In [ ]:
raw_seq = pd.read_csv("../../evofr/test/testing_data/mlr-variant-counts.tsv", sep="\t")
raw_seq = raw_seq[raw_seq.location == "City0"]
data = ef.VariantFrequencies(raw_seq, pivot="C")

In [ ]:
from evofr.plotting import FrequencyPlot, GrowthAdvantagePlot, TimeVaryingPlot, PatchLegend

freq_plot = FrequencyPlot(data=data).plot(posterior=False);
PatchLegend(color_map=freq_plot.color_map).add_legend(ax=freq_plot.ax);

In [ ]:
from relative_fitness_gp import SquaredExpCov, RelativeFitnessGP

In [ ]:
kernel = SquaredExpCov()

model = RelativeFitnessGP(SquaredExpCov(alpha=0.1, rho=90)) # Do I need to add intercepts?
inference_method = ef.InferNUTS(num_samples=500, num_warmup=500)
#inference_method = ef.InferMAP(lr=4e-4, iters=50_000)

posterior = inference_method.fit(model, data=data)

In [ ]:
freq_plot = FrequencyPlot(posterior).plot();

In [ ]:
import jax.numpy as jnp
from tinygp import kernels, GaussianProcess
from jax.nn import softmax
from jax.scipy.special import logit

def forecast(model, samples, forecast_L):
        # Create time points to forecast
        N_samples, N_time, N_variant = samples["delta"].shape
        ts = jnp.arange(N_time)
        pred_ts = np.arange(1, forecast_L+1) + N_time

        # Define GP
        # Need to condition on samples though right?
        gp = GaussianProcess(model.gp.model_numpyro(), ts, diag=1e-6)

        # Forecast relative fitness
        def _condition_sample(fitness, xs, key):
            gp_cond = gp.condition(fitness, xs)
            return gp_cond.gp.sample(key)

        ## Map over sample and variant
        condition_sample = jax.vmap(
            jax.vmap(_condition_sample, in_axes=(1, None, None), out_axes=1),
            in_axes=(0, None, 0),
            out_axes=0,
        )

        keys = jax.random.PRNGKey(1)
        keys = jax.random.split(keys, N_samples)
        samples["delta_forecast"] = condition_sample(samples["delta"], pred_ts, keys)
        
        # Forecast frequency
        def _forecast_freq(fitness, frequency):
            init_freq = jnp.log(frequency[-1, :]) # Last known frequency
            _fitness = jnp.concatenate((fitness, jnp.zeros((forecast_L, 1))), axis=-1)
            cum_fitness = jnp.cumsum(_fitness.at[0,:].set(0), axis=0)
            return softmax(cum_fitness + init_freq, axis=-1)
            
        vmap_forecast_freq = jax.vmap(_forecast_freq, in_axes=(0,0), out_axes=0)
        samples["freq_forecast"] = vmap_forecast_freq(samples["delta_forecast"], samples["freq"])
        return samples

In [ ]:
posterior.samples = forecast(model, posterior.samples, 30)

In [ ]:
freq_plot = FrequencyPlot(posterior).plot(forecast=True);

In [ ]:
delta_plot = TimeVaryingPlot(posterior=posterior, site="delta").plot()
delta_plot.ax.set_ylabel("Relative Fitness")

In [ ]:
delta_plot = TimeVaryingPlot(posterior=posterior, site="delta_forecast").plot()
delta_plot.ax.set_ylabel("Relative Fitness")

In [ ]:
# How do I deal with nans here?

In [ ]:
# How do I want this to be used?

# Basically, if you specify a kernel you can easily estimate the relative fitnesses and plot forecasts with uncertainty
# The difficulties are specifying this diag?
# I wonder if I'm better off reimplementing a lot of this myself. 